In [ ]:
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
import string
from nltk.stem import WordNetLemmatizer

In [ ]:
data=pd.read_csv('../input/nlp-getting-started/train.csv',index_col=0)
test=pd.read_csv('../input/nlp-getting-started/test.csv',index_col=0)

In [ ]:
data.fillna('',inplace=True)

In [ ]:
test.fillna('',inplace=True)

In [ ]:
data.head()

In [ ]:
test.head()

In [ ]:
finaldata=data['keyword']+' '+data['location']+' '+data['text']
testdata=test['keyword']+' '+test['location']+' '+test['text']
finaltarget=data['target']

In [ ]:
finaldata.head()

In [ ]:
testdata.head()

In [ ]:
finaltarget.head()

In [ ]:
WNL=WordNetLemmatizer()
def text_process(data):
    msg=[c for c in data if c not in string.punctuation]
    msg=''.join(msg)
    msg=[word for word in msg.split() if word.lower() not in stopwords.words('english')]
    msg=[WNL.lemmatize(word) for word in msg ]
    return msg

In [ ]:
finaldata.head(10).apply(text_process)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfTransformer

In [ ]:
CV=CountVectorizer(analyzer=text_process)

In [ ]:
CV.fit(finaldata)

In [ ]:
CV.vocabulary_

In [ ]:
finaldata=CV.transform(finaldata)

In [ ]:
finaldata.nnz

In [ ]:
Tfidf=TfidfTransformer()
Tfidf.fit(finaldata)
Tfidf_val=Tfidf.transform(finaldata)

In [ ]:
testdata=CV.transform(testdata)
testdata=Tfidf.transform(testdata)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB

# Naive Bayes & Random Forest

In [ ]:
sentiment=MultinomialNB()
sentiment.fit(Tfidf_val,finaltarget)
result1=sentiment.predict(testdata)

RFC=RandomForestClassifier()
RFC.fit(Tfidf_val,finaltarget)
result2=RFC.predict(testdata)

In [ ]:
rslt=pd.DataFrame(index=test.index)
rslt['Naive Bayes']=result1
rslt['Random Forest']=result2

In [ ]:
rslt['Naive Bayes'].value_counts()

In [ ]:
rslt['Random Forest'].value_counts()

# Support Vector Machine

In [ ]:
from sklearn.svm import SVC

In [ ]:
svc=SVC()

In [ ]:
svc.fit(Tfidf_val,finaltarget)
svm=svc.predict(testdata)

In [ ]:
rslt['svm']=svm

In [ ]:
rslt['svm'].value_counts()

### With Grid Search

In [ ]:
from sklearn.model_selection import GridSearchCV
grid={'C':[0.01,0.1,1,10,100],'gamma':[0.001,.01,0.1,1,10]}

In [ ]:
g=GridSearchCV(SVC(),grid,verbose=2)

In [ ]:
g.fit(Tfidf_val,finaltarget)

In [ ]:
g.best_estimator_

In [ ]:
grid=g.predict(testdata)

In [ ]:
rslt['grid']=grid

In [ ]:
rslt['grid'].value_counts()

In [ ]:
rslt.head()

In [ ]:
#rslt.to_csv('prediction.csv')